###  Bagging   如雷贯耳，投票类算法的代表

###  bagging的思路
  与投票法不同的是，Bagging不仅仅集成模型最后的预测结果，**同时采用一定策略来影响基模型训练，保证基模型可以服从一定的假设**。在上一章中我们提到，**希望各个模型之间具有较大的差异性**，而在实际操作中的模型却往往是同质的，因此**一个简单的思路是通过不同的采样增加模型的差异性**。

**自采样法，样本扰动和随机选取部分而不是全部变量进行分裂点计算，是随机森林效果好的原因。**

### bagging的原理分析
  Bagging的核心在于自助采样(bootstrap)这一概念，即有放回的从数据集中进行采样，也就是说，同样的一个样本可能被多次进行采样。一个自助采样的小例子是我们希望估计全国所有人口年龄的平均值，那么**我们可以在全国所有人口中随机抽取不同的集合（这些集合可能存在交集），计算每个集合的平均值，然后将所有平均值的均值作为估计值。**

  首先我们随机取出一个样本放入采样集合中，再把这个样本放回初始数据集，重复K次采样，最终我们可以获得一个大小为K的样本集合。同样的方法， 我们可以采样出T个含K个样本的采样集合，然后基于每个采样集合训练出一个基学习器，再将这些基学习器进行结合，这就是Bagging的基本流程。

  对回归问题的预测是通过预测取平均值来进行的。对于分类问题的预测是通过对预测取多数票预测来进行的。**Bagging方法之所以有效，是因为每个模型都是在略微不同的训练数据集上拟合完成的，这又使得每个基模型之间存在略微的差异，使每个基模型拥有略微不同的训练能力**。

  Bagging同样是一种降低方差的技术，**因此它在不剪枝决策树、神经网络等易受样本扰动的学习器上效果更加明显**。在实际的使用中，加入列采样的**Bagging技术对高维小样本往往有神奇的效果**。

###  bagging的案例分析(基于sklearn，介绍随机森林的相关理论以及实例)
  Sklearn为我们提供了 BaggingRegressor 与 BaggingClassifier 两种Bagging方法的API，我们在这里通过一个完整的例子演示Bagging在分类问题上的具体应用。这里两种方法的默认基模型是树模型。

  这里的树模型一般指决策树，它是一种树形结构，树的每个非叶子节点表示对样本在一个特征上的判断，节点下方的分支代表对样本的划分。决策树的建立过程是一个对数据不断划分的过程，每次划分中，首先要选择用于划分的特征，之后要确定划分的方案（类别/阈值）。我们希望通过划分，**决策树的分支节点所包含的样本“纯度”尽可能地高**。节点划分过程中所用的指标主要是信息增益和GINI系数。

**注：一个树的节点里面只包含一种类别，无疑是纯度最高的。**

&emsp;&emsp;信息增益衡量的是划分前后信息不确定性程度的减小。信息不确定程度一般使用信息熵来度量，其计算方式是：
$$
H(Y) = -\sum{p_ilogp_i}
$$
&emsp;&emsp;其中i表示样本的标签，p表示该类样本出现的概率。当我们对样本做出划分之后，计算样本的条件熵：
$$
H(Y|X) = -\sum_{x\in X}p(X=x){H(Y|X=x)}
$$

&emsp;&emsp;其中x表示用于划分的特征的取值。信息增益定义为信息熵与条件熵的差值：
$$
IG = H(Y) - H(Y|X)
$$

&emsp;&emsp;信息增益IG越大，说明使用该特征划分数据所获得的信息量变化越大，子节点的样本“纯度”越高。

&emsp;&emsp;同样的，我们也可以利用Gini指数来衡量数据的不纯度，计算方法如下：
$$
Gini = 1 - \sum{p_i^2}
$$
&emsp;&emsp;当我们对样本做出划分后，计算划分后的Gini指数：
$$
Gini_x = \sum_{x\in X}p(X=x)[1 - \sum{p_i^2}]
$$
&emsp;&emsp;一般来说，我们选择使得划分后Gini指数最小的特征（注意这里是直接根据Gini指数进行判断，而并非其变化量）。

**注：感觉西瓜书里面的公式，更加清晰一点。**

&emsp;&emsp;下方给出了决策树的一个例子，我们要训练一个模型，根据天气、温度和风力等级来判断是否打网球。    
![image-20210411151227186](./image/DecisionTree.png)

首先我们通过计算信息增益或Gini指数确定了首先根据天气情况对样本进行划分，之后对于每个分支，继续考虑除天气之外的其他特征，直到样本的类别被完全分开，所有特征都已使用，或达到树的最大深度为止。

Bagging的一个典型应用是随机森林。顾名思义，“森林”是由许多“树”bagging组成的。在具体实现上，用于每个决策树训练的样本和构建决策树的特征都是通过随机采样得到的，随机森林的预测结果是多个决策树输出的组合（投票）。随机森林的示意图如下：

![image-20210411161043251](./image/RandomForest.png)



下面我们使用sklearn来实现基于决策树方法的bagging策略。

In [1]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15,    # 这里和上节是一样的
                           n_redundant=5, random_state=5)
print(X.shape, y.shape)

(1000, 20) (1000,)


我们将使用重复的分层k-fold交叉验证来评估该模型，一共重复3次，每次有10个fold。我们将评估该模型在所有重复交叉验证中性能的平均值和标准差。

In [2]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier()   # 内部是什么算法。。
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.857 (0.043)


In [4]:
BaggingClassifier??

In [8]:
from sklearn.svm import SVC, LinearSVC

model = BaggingClassifier(base_estimator=SVC())   
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Accuracy: 0.916 (0.028)


###  发现使用 SVC 效果更好

## 总结下：这里讲得不够深入，随机森林没有展开讲。。希望做得更好！加油！